<a href="https://colab.research.google.com/github/sivasankar3002/Automated-Python-Docstring-Generator/blob/Milestone-1/Automated_Python_Docstring_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install gradio

In [10]:
import ast
import json
import gradio as gr
import tempfile


def generate_stub(node):
    args = []
    for arg in node.args.args:
        if arg.arg != "self":
            args.append(arg.arg)

    stub = '"""\nTODO: Add description.\n\n'
    if args:
        stub += "Args:\n"
        for a in args:
            stub += f"    {a}: Description.\n"

    stub += "\nReturns:\n    Description.\n\"\"\""
    return f"{node.name}:\n{stub}"


def analyze_python_file(file):
    if file is None:
        return "", "", "", "", "", "", "", "", None

    with open(file.name, "r", encoding="utf-8") as f:
        source_code = f.read()

    try:
        tree = ast.parse(source_code)
    except SyntaxError as e:
        return f"Syntax Error:\n{e}", "", "", "", "", "", "", "", None

    functions = []
    classes = []
    documented = []
    undocumented = []
    stubs = []

    for node in ast.walk(tree):

        if isinstance(node, ast.FunctionDef):
            functions.append(node.name)
            if ast.get_docstring(node):
                documented.append(f"Function: {node.name}")
            else:
                undocumented.append(f"Function: {node.name}")
                stubs.append(generate_stub(node))

        elif isinstance(node, ast.ClassDef):
            classes.append(node.name)
            if ast.get_docstring(node):
                documented.append(f"Class: {node.name}")
            else:
                undocumented.append(f"Class: {node.name}")
                stubs.append(generate_stub(node))


    total_functions = len(functions)
    total_classes = len(classes)
    total_objects = total_functions + total_classes

    parser_accuracy = (total_objects / total_objects * 100) if total_objects > 0 else 0


    functions_text = "\n".join(f"- {f}" for f in functions) or "No functions found."
    classes_text = "\n".join(f"- {c}" for c in classes) or "No classes found."
    documented_text = "\n".join(f"- {d}" for d in documented) or "No existing docstrings."
    undocumented_text = "\n".join(f"- {u}" for u in undocumented) or "No missing docstrings."
    stubs_text = "\n\n".join(stubs) or "No stubs generated."

    stats_text = (
        f"Total Functions: {total_functions}\n"
        f"Total Classes: {total_classes}"
    )

    accuracy_text = f"Parser Accuracy: {parser_accuracy:.2f}%"

    coverage_text = (
        f"Total Objects: {total_objects}\n"
        f"Documented: {len(documented)}\n"
        f"Missing Docstrings: {len(undocumented)}"
    )


    report = {
        "total_functions": total_functions,
        "total_classes": total_classes,
        "documented": documented,
        "missing_docstrings": undocumented,
        "parser_accuracy": parser_accuracy
    }

    temp = tempfile.NamedTemporaryFile(delete=False, suffix=".json", mode="w", encoding="utf-8")
    json.dump(report, temp, indent=4)
    temp.close()

    return (
        functions_text,
        classes_text,
        documented_text,
        undocumented_text,
        stubs_text,
        stats_text,
        accuracy_text,
        coverage_text,
        temp.name
    )



with gr.Blocks(css="""
.section {border: 1px solid #ddd; padding: 15px; border-radius: 10px; margin-bottom: 15px;}
""") as demo:

    gr.Markdown("# 🐍 Automated Python Docstring Generator (Milestone-1)")

    file_input = gr.File(label="📂 Upload Python File (.py)", file_types=[".py"])

    with gr.Group(elem_classes="section"):
        gr.Markdown("## 🔍 Code Structure")
        with gr.Row():
            functions_box = gr.Textbox(label="Functions Found", lines=6)
            classes_box = gr.Textbox(label="Classes Found", lines=6)

    with gr.Group(elem_classes="section"):
        gr.Markdown("## 📄 Docstring Status")
        with gr.Row():
            documented_box = gr.Textbox(label="Existing Docstrings", lines=6)
            undocumented_box = gr.Textbox(label="Missing Docstrings", lines=6)

    with gr.Group(elem_classes="section"):
        gr.Markdown("## 📝 Generated Baseline Docstring Stubs")
        stubs_box = gr.Textbox(lines=12)

    with gr.Group(elem_classes="section"):
        gr.Markdown("## 📊 Statistics & Accuracy")
        stats_box = gr.Textbox(label="Object Statistics", lines=3)
        accuracy_box = gr.Textbox(label="Parser Accuracy", lines=2)
        coverage_box = gr.Textbox(label="Coverage Summary", lines=4)
        download_box = gr.File(label="⬇️ Download Coverage Report (JSON)")

    file_input.change(
        analyze_python_file,
        inputs=file_input,
        outputs=[
            functions_box,
            classes_box,
            documented_box,
            undocumented_box,
            stubs_box,
            stats_box,
            accuracy_box,
            coverage_box,
            download_box,
        ]
    )

demo.launch()


/tmp/ipython-input-4129221724.py:115: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css="""


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d36909cc5bf114e209.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [3]:
pip install pytest

In [8]:
!pytest test_ast_analysis.py


============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0
rootdir: /content
plugins: typeguard-4.4.4, anyio-4.12.0, langsmith-0.4.59
collected 4 items                                                              

test_ast_analysis.py ....                                                [100%]

============================== 4 passed in 0.02s ===============================
